# Sandbox

> The goal of this notebook is to play around with the `BAM`/`SAM`, `BCF`/`VCF`, and `FASTA`/`FASTQ` file.  It's been years since I've done any variant calling, and I'm curious to see how well I can work with them in Python.

__Tutorials__:

- [Working with Genomic Data in Python](http://fullstackdatascientist.io/15/03/2016/genomic-data-visualization-using-python/)
- [Extracting Data Files from VCF](http://alimanfoo.github.io/2017/06/14/read-vcf.html)
- [Framework for Evaluating Variant Detection Methods](https://bcb.io/2013/05/06/framework-for-evaluating-variant-detection-methods-comparison-of-aligners-and-callers/)

## Tools

__Database Access__:

- ~~[cruzdb](https://github.com/brentp/cruzdb): UCSC genomes database~~ 
- [pyensembl](https://github.com/openvax/pyensembl): Ensembl
- ~~[bioservices](https://github.com/cokelaer/bioservices): ArrayExpress, BioModels, ChEBI, KEGG, PDB, Uniprot, UniChem, NCBIBlast~~

__File Handling__:
 
- [bamnostic](https://github.com/betteridiot/bamnostic): `BAM`
- [pysamstats](https://github.com/alimanfoo/pysamstats): `BAM` _commandline tool_
- [pyVCF](https://github.com/jamescasbon/PyVCF): `VCF`
- [pyfaidx](https://github.com/mdshw5/pyfaidx): `FASTA`
- [fastq-and-furious](https://github.com/lgautier/fastq-and-furious): `FASTQ`
- ~~[cyvcf2](https://github.com/brentp/cyvcf2): `VCF` and `BCF`~~ 
- [pysam](https://github.com/pysam-developers/pysam): `SAM`, `BAM`, `VCF`, and `BCF`
- [pybedtools](https://github.com/daler/pybedtools): `BAM`, `SAM`, `BED`, `BCF`, `VCF`, `GFF`, and `GTF` 
- [htseq](https://github.com/simon-anders/htseq): `FASTQ`, `BAM`, `SAM`, `VCF`, `BED`
- [biopython](https://github.com/biopython/biopython): DNA, RNA, Proteins, `FASTQ`
- [scikit-allel](https://github.com/cggh/scikit-allel): `VCF`

* trouble installing cyVCF2

__Quality Control__:

- ~~[AfterQC](https://github.com/OpenGene/AfterQC): `FASTQ` file Processing~~
- [fastqp](https://github.com/mdshw5/fastqp): `FASTQ`, `BAM`, and `SAM` file Processing
- ~~[bam-toolbox](https://github.com/AndersenLab/bam-toolbox) `BAM` coverage~~
- ~~[mergesam](https://github.com/DarwinAwardWinner/mergesam): `SAM` and `BAM` conversion and merging and sorting~~

__Variant Calling__:

- [freebayes](https://github.com/ekg/freebayes): a haplotype-based variant detector
- [gatk](https://software.broadinstitute.org/gatk/): assortment of variant callers
- [bcftools](http://www.htslib.org/): assortment of variant callers

__Visualization__:

- ~~[Biodalliance](https://github.com/dasmoth/dalliance): visualize `BAM`, `VCF`~~
- [GenomeView](https://github.com/nspies/genomeview): visualize `BAM`
- [plotly](https://github.com/plotly): general-purpose graphing library
- [seaborn](https://github.com/mwaskom/seaborn): general-purpose graphing library
- [altair](https://github.com/altair-viz/altair): statistical visualization

__Report Generation__:

- [MultiQC](https://github.com/ewels/MultiQC): Aggregate results from bioinformatics analyses across many samples into a single report
- [Jupyter Notebook](https://github.com/jupyter/notebook)
- [pyPipeline](https://github.com/AndersenLab/pyPipeline)
- [bcbio](https://github.com/bcbio/bcbio-nextgen)

__Math/Stat__:

- [pandas](https://github.com/pandas-dev/pandas)
- [scipy](https://github.com/scipy/scipy)
- [cython](https://github.com/cython/cython)

__Tutorials__:

- [samtools tutorial](http://quinlanlab.org/tutorials/samtools/samtools.html)
- [ngs alignment and variant calling](https://github.com/ekg/alignment-and-variant-calling-tutorial)
- [scikit-allel](http://alimanfoo.github.io/2017/06/14/read-vcf.html)
- [genomic data visualization](http://fullstackdatascientist.io/15/03/2016/genomic-data-visualization-using-python/)

#### Let's sanity check out packages.

In [ ]:
!pip freeze

#### Let's import what we need.

In [ ]:
import os
import glob
import time
import itertools
from collections import OrderedDict 

import numpy as np
import pandas as pd

import pysam as ps
import vcf
import bamnostic as bs
import pybedtools as bt
import HTSeq as hs
import fastqandfurious
from Bio import SeqIO
import allel

import seaborn as sns
sns.set()

Let's define __PATHs__ for our data files.

In [ ]:
aln1_path = "/home/ifrancium/Documents/amplicon-ngs-workflow/notebooks/data/original_data/aln1.fastq"
aln2_path = "/home/ifrancium/Documents/amplicon-ngs-workflow/notebooks/data/original_data/aln2.fastq"

In [ ]:
bam_path = "/home/ifrancium/Documents/amplicon-ngs-workflow/notebooks/data/samtools_output/aln_sorted.bam"
bai_path = "/home/ifrancium/Documents/amplicon-ngs-workflow/notebooks/data/samtools_output/aln_sorted.bam.bai"

In [ ]:
sam_path = "/home/ifrancium/Documents/amplicon-ngs-workflow/notebooks/data/samtools_output/aln.sam"

In [ ]:
gtf3_path = "/home/ifrancium/Documents/amplicon-ngs-workflow/reference_sequences/Homo_sapiens.GRCh37/Homo_sapiens.GRCh37.87.gtf"

In [ ]:
dump_path = "/home/ifrancium/Documents/amplicon-ngs-workflow/notebooks/data/jupyter_outputs/"

#### Let's take a look inside the FASTQ files.

In [ ]:
aln1 = hs.FastqReader(aln1_path)
aln2 = hs.FastqReader(aln2_path)

In [ ]:
print("There are ", sum(1 for x in aln1), " reads in the aln1 FASTQ file.")
print("There are ", sum(1 for x in aln2), " reads in the aln2 FASTQ file.")

`aln1` indicates the forward reads, and `aln2` indicates the reverse reads.  I'm a big fan of `pandas.DataFrame`, so let's try to mash everything in there.

In [ ]:
sample_size = 575002

t_start = time.time()

print("Printing the first {} reads in aln1:".format(sample_size))

aln1_fastq_reads = []

for cnt, fastq_read in enumerate(itertools.islice(aln1, sample_size)):
    read_dict = {"Name": fastq_read.name, "Sequence": str(fastq_read), "Phred Score": fastq_read.qual, "Length": len(fastq_read)}
    aln1_fastq_reads.append(read_dict)  

    print(cnt, "Read Length: ", len(fastq_read), ", Average Phred Score: ", np.sum(fastq_read.qual)/len(fastq_read))


# construct dataframe
aln1_fastq_df = pd.DataFrame(aln1_fastq_reads)

t_stop = time.time()
t_total = t_stop - t_start

In [ ]:
head = 5

print("There are ", len(aln1_fastq_df.index), " reads in this dataframe.")
print("This job took {} seconds to complete.".format(t_total))
print("Printing the first {} rows of the dataframe:".format(head))
aln1_fastq_df.head(head)

Let's back up this DataFrame.

In [ ]:
aln1_fastq_df.to_csv(dump_path + "aln1_FASTQ.csv")

Let's move onto the `aln2` FASTQ file.

In [ ]:
sample_size = 575002

t_start2 = time.time()

print("Printing the first {} reads in aln2:".format(sample_size))

aln2_fastq_reads = []

for cnt, fastq_read in enumerate(itertools.islice(aln2, sample_size)):
    read_dict = {"Name": fastq_read.name, "Sequence": str(fastq_read), "Phred Score": fastq_read.qual, "Length": len(fastq_read)}
    aln2_fastq_reads.append(read_dict)  

    print(cnt, "Read Length: ", len(fastq_read), ", Average Phred Score: ", np.sum(fastq_read.qual)/len(fastq_read))

# construct dataframe
aln2_fastq_df = pd.DataFrame(aln2_fastq_reads)

t_stop2 = time.time()
t_total2 = t_stop2 - t_start2

In [ ]:
head2 = 5

print("There are ", len(aln2_fastq_df.index), " reads in this dataframe.")
print("This job took {} seconds to complete.".format(t_total2))
print("Printing the first {} rows of the dataframe:".format(head2))
aln2_fastq_df.head(head)

Let's back up this DataFrame.

In [ ]:
aln2_fastq_df.to_csv(dump_path + "aln2_FASTQ.csv")

#### Let's take a look inside the BAM file.

In [ ]:
aln = hs.BAM_Reader(bam_path)

In [ ]:
print("There are ", sum(1 for x in aln), " alignment reads in this BAM file.")

In [ ]:
print(type(aln))

We need to know all the methods of this object: `<class 'HTSeq.BAM_Reader'>`.  Let's start out with the alignment reads data.  These reads are `HTSeq.SAM_Alignment` objects.

#### `HTSeq.SAM_Alignment` Object Attributes

> Note: I put `(iterable)` in with some of the Return Types.  This is not accurate and a bit confusing.  This is meant to convey that the dunder functions returned a string (or whatever object type is specified) to the terminal, but the return type is actually a unique class object, which contains that particular type object returned by the dunder function.   

| Name | Return Type | Description |
|------|-------------|-------------|
| `.read` | str (iterable) | A Sequence object (str) holds a DNA sequence.  May have a `descr` (str) attribute. |
| `.aligned` | bool | `True` if aligned, `False` if not aligned.  This is determined by the aligner. |
| `.iv` | list (iterable) | contains `chrom` (str), `start` (int), `end` (int), and `strand` (str) in the following format: `chrom:start-end/strand` |
| `.paired_end` | bool | `True` if read stems from a paired-end sequencing run, and `False` if not. |
| `.read_as_aligned` | str (iterable) | `read` attribute always presents the read in the order in which it was sequenced. This attribute undoes the reverse-complement (if performed at all), and the `read` attribute is as it was found in the file. | 
| `.read_as_sequenced` | str (iterable) | This attribute performs a reverse-complement on the sequencing read if the aligner reverse-complemented the sequencing read to align it to the (+) strand. |
| `.aQual` | int | The alignment quality score in Phread style encoding. |
|`.cigar` | list | A list of `CigarOperation` objects, as parsed from the extended CIGAR string. These objects have the following attributes: `type`, `size`, `ref_iv`, `query_from`, `query_to`, `check()`. |
| `.not_primary_alignment` | bool | Returns `True` if `0x100` flag is present during alignment.  This is indicative of secondary alignment, which refers to multiple alignments. |
| `.failed_platform_qc` | bool | Returns `True` if the read failed a platform quality check.  This is accompanied by a `0x200` flag. |
| `.pcr_or_optical_duplicate` | bool | Returns `True` if the read is a PCR or optical duplicate.  This is accompanied by a `0x400` flag.
| `.supplementary` | bool | Returns `True` if `0x800` flag is present during alignment.  This suggests chimeric/non-linear alignments. | 
| `.optional_field(tag)` | str | Returns the optional field `tag`, which are two-letter strings found at the beginning of the header lines. | 
| `.optional_fields` | dict | Returns a dict with all optional fields, using their `tags` as `keys`. | 
| `.get_sam_lines()` | str | Constructs a SAM line to describe the alignment |
| `.mate_aligned` | bool | Only available if `.paired_end == True`.  Returns `true` if the pair (__mates__) was aligned. |
| `.pe_which` | str | Only available if `.paired_end == True`.  Takes one of the values `first`, `second`, `unknown` and `not_paired_end`, to indicate whether the read stems from the first or second pass of the paired-end sequencing. |
| `.proper_pair` | Bool | Only available if `.paired_end == True`.  Returns `True` if the mates formed a proper alignment with each other and to the reference.  This is indicated by the `0x0002` flag. |
| `.mate_start` | int (iterable) | Only available if `.paired_end == True`.  The start (i.e., left-most position) of the mate’s alignment. Note that `mate_start.strand` is opposite to `iv.strand` for proper pairs. |
| `.inferred_insert_size` | int | Only available if `.paired_end == True`.  Denotes the zize of the insert between the reads.  If the reads overlap, `.inferred_insert_size` would be negative. |


#### `SAM`/`BAM` Alignment Attributes

> As per the [working](https://samtools.github.io/hts-specs/SAMv1.pdf) specification.

| Col |	Field |	Type | Brief Description | Accessible through `HTSeq`? |
|-----|-------|------|-------------------|-----------------------------|
| 1 | QNAME | String | Query template NAME | No |
| 2 | FLAG | Int bitwise | FLAG | Some |
| 3 | RNAME | String | References sequence NAME | No |
| 4 | POS | Int | 1- based leftmost mapping POSition | Yes |
| 5 | MAPQ | Int | MAPping Quality | Yes |
| 6 | CIGAR | String | CIGAR String | Yes | 
| 7 | RNEXT | String | Ref. name of the mate/next read | No (?) |
| 8 | PNEXT | Int | Position of the mate/next read | Yes |
| 9 | TLEN | Int | observed Template LENgth | No | 
| 10 | SEQ | String | segment SEQuence | Yes |
| 11 | QUAL | String | ASCII of Phred-scaled base QUALity+33 | Yes | 

#### The `FLAG` field types

| Bit | Description |
|-----|-------------|
| 0x1 | template having multiple segments in sequencing |
| 0x2 | each segment properly aligned according to the aligner |
| 0x4 | segment unmapped |
| 0x8 | next segment in the template unmapped |
| 0x10 | SEQ being reverse complemented |
| 0x20 | SEQ of the next segment in the template being reverse complemented |
| 0x40 | the first segment in the template | 
| 0x80 | the last segment in the template |
| 0x100 | secondary alignment |
| 0x200 | not passing filters, such as platform/vendor quality controls |
| 0x400 | PCR or optical duplicate |
| 0x800 | supplementary alignment |


In [ ]:
sample_size = 1150004

t_start3 = time.time()

print("Printing the first {} reads in aln:".format(sample_size))

aln_alignment_reads = []

for cnt, alignment_read in enumerate(itertools.islice(aln, sample_size)):
    read_dict = {
        "Sequence": str(alignment_read.read), 
        "Aligned?": alignment_read.aligned,
        "Coverage": alignment_read.iv,
        # "Paired-end?": alignment_read.paired-end, # This could probably be removed. NOT WORKING
        "Phred Score": alignment_read.aQual,
        "CIGAR Objects": alignment_read.cigar,
        "Secondary Alignment (0x100)": alignment_read.not_primary_alignment,
        "Failed QC? (0x200)": alignment_read.failed_platform_qc,
        "PCR/Optical duplicate? (0x400)": alignment_read.pcr_or_optical_duplicate,
        "Chimeric Alignment (0x800)": alignment_read.supplementary,
        # "Alignment Description": alignment_read.get_sam_lines(), # NOT WORKING
        "Pairs Overlap?": alignment_read.mate_aligned,
        "Both Pairs Aligned? (0x0002)": alignment_read.proper_pair,
        "Which Pair Aligned?": alignment_read.pe_which,
        "Alignment Start Position": alignment_read.mate_start, # should be the same as `.iv.start`
        "Pair Gap Distance": alignment_read.mate_start,
        }
    aln_alignment_reads.append(read_dict)  

    print(cnt, "Read Length: ", len(fastq_read), "Average Phred Score: ", np.sum(fastq_read.qual)/len(fastq_read))

# construct dataframe
aln_df = pd.DataFrame(aln_alignment_reads)

t_stop3 = time.time()
t_total3 = t_stop3 - t_start3

In [ ]:
head3 = 5
print("There are ", len(aln_df.index), " reads in this dataframe.")
print("This job took {} seconds to complete.".format(t_total3))
print("Printing the first {} rows of the dataframe:".format(head3))
aln_df.head(head)

Let's grab the experiment meta-data invoking the `.optional_fields` method.

__Some Thoughts__: Looking at how the `HTSeq.par_SAM_alignments` superclass is designed, I'm confused why the authors didn't just architect the superclass as described in the `SAM/BAM` format specifications.  I should be able to invoke a method to call whichever `flag` value is present in column 2 of the BAM Alignment section.  Instead, I'll have to either convert to the `SAM` format, or parse binary.  Also, `Pair Gap Distance` seems to be calling `Alignment Start Position`.  It's OK, It can be calculated from `Coverage`.

#### Evaluating Coverage

In [ ]:
gff_file = hs.GFF_Reader(gff_path)

In [ ]:
coverage = hs.GenomicArray("auto")

So `coverage`, which contains our read counts, is an `HTSeq._HTSeq.GenomicArray` class object, which is a dictionary for `HTSeq._HTSeq.GenomicInterval` class objects.  The `Coverage` column in `aln_df`, is also a `HTSeq._HTSeq.GenomicInterval` class object.  

In [ ]:
sample_size = 1150004
# 1150004

aligned_total = 0

t_start4 = time.time()

for cnt, alignment_read in enumerate(itertools.islice(aln, sample_size)):
    if alignment_read.aligned == True:
        coverage[alignment_read.iv] += 1
        aligned_total += 1

not_aligned = sample_size - aligned_total

t_stop4 = time.time()
t_total4 = t_stop4 - t_start4

In [ ]:
print("It took {} seconds to scrape 1 column from aln_df.".format(t_total4))
print("Our BAM file contains {} sequencing reads that were classified as ALIGNED!".format(aligned_total))
print("Only {} reads failed to align correctly.".format(not_aligned))

In [ ]:
# Let's see what our coverage dictionary looks like

for location, count in coverage.steps():
    print("There are ", count, " aligned reads at the following location: ", location, ".")    

#### Finding Transcription Start Sites

To find the location of all transcription start sites, we can look in the `GFF` file for exons with `exon_number == 1` (as indicated by the `exon_number` attribute in Ensembl GFF files) and ask for their directional start (`start_d`). 

In [ ]:
for feature in itertools.islice(gff_file, 10):
    print(feature.type)
    for key, value in feature.attr.items():
        print("\n", key, "||||", value)
    # if feature.type == "exon" and feature.attr["exon_number"] == "1":
        
        # print(feature.attr["gene_id"], feature.attr["transcript_id"], feature.iv.start_d_as_pos)

#### SAM/BAM Global Attributes



#### Converting BAM to SAM